# SMAP-USDA monthly averages

Monthly climatologies/stats of 
soil moisture quantities and 
their derivatives (porosity and available porosity)

James McCreight 


In [1]:
from bokeh.io import output_notebook 
from bokeh.plotting import figure, output_file, show 
from bokeh.palettes import brewer, colorblind
import ee

In [3]:
ee.Authenticate()

Enter verification code:  4/1AY0e-g50N5cz0ku9Lik0TySKyPKyg2o_vYHcr4YDpbL8Fee2xVW4YE2iqzk



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
smap_usda = (ee
  .ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture'))

In [6]:
bands = ee.List(['smp', 'ssm', 'susm'])
months = ee.List.sequence(1, 12);

In [7]:
# monthly averages for each band
def band_mon_avg(bb):
  def mon_avg(mm):
    return (smap_usda
           .filter(ee.Filter.calendarRange(mm, mm, 'month'))
           .select([bb])
           .mean()
           .set('month', mm).set('band', bb))
  return ee.ImageCollection.fromImages(months.map(mon_avg).flatten())

In [8]:
monthly_means = bands.map(band_mon_avg)

In [9]:
print(monthly_means)

ee.List({
  "functionInvocationValue": {
    "functionName": "List.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_2_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "ImageCollection.fromImages",
              "arguments": {
                "images": {
                  "functionInvocationValue": {
                    "functionName": "List.flatten",
                    "arguments": {
                      "list": {
                        "functionInvocationValue": {
                          "functionName": "List.map",
                          "arguments": {
                            "baseAlgorithm": {
                              "functionDefinitionValue": {
                                "argumentNames": [
                                  "_MAPPING_VAR_1_0"
                                ],
                                "bod

In [10]:
monthly_means.getInfo()

[{'type': 'ImageCollection', 'bands': []},
 {'type': 'ImageCollection', 'bands': []},
 {'type': 'ImageCollection', 'bands': []}]

In [11]:
# separate the variables
smp = ee.ImageCollection(monthly_means.get(0))
ssm = ee.ImageCollection(monthly_means.get(1))
susm = ee.ImageCollection(monthly_means.get(2))

In [12]:
# porosity in mm (millimeters)
def mon_porosity_calc(mm):
  porosity = (ssm
    .filter(ee.Filter.eq('month', mm)).first()
    .add(susm.filter(ee.Filter.eq('month', mm)).first())
    .divide(smp.filter(ee.Filter.eq('month', mm)).first())
    .set('month', mm).set('band', 'porosity_mm'))
  return porosity

porosity = ee.ImageCollection(months.map(mon_porosity_calc))
porosity.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 1, 'band': 'porosity_mm', 'system:index': '0'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 2, 'band': 'porosity_mm', 'system:index': '1'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 3, 'band': 'porosity_mm', 'system:index': '2'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
  

In [13]:
# available/additional/unused porosity
def mon_avail_porosity_calc(mm):
  avail_porosity = (porosity
    .filter(ee.Filter.eq('month', mm)).first()
    .subtract(
      ssm.filter(ee.Filter.eq('month', mm)).first()
      .add(susm.filter(ee.Filter.eq('month', mm)).first()))
      .set('month', mm).set('band', 'avail_porosity_mm'))
  return avail_porosity

avail_porosity = ee.ImageCollection(months.map(mon_avail_porosity_calc))
avail_porosity.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 1,
    'band': 'avail_porosity_mm',
    'system:index': '0'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 2,
    'band': 'avail_porosity_mm',
    'system:index': '1'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 3,
    'band': 'avail_porosity_mm',
    'system:index': '2'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
    

In [14]:
# Export to images, build a collection
smap_collection = ee.ImageCollection(
  ee.List([
    ssm.toList(12),
    susm.toList(12),
    smp.toList(12),
    porosity.toList(12),
    avail_porosity.toList(12)]).flatten())

smap_collection.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 1, 'band': 'ssm', 'system:index': '0'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 2, 'band': 'ssm', 'system:index': '1'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month': 3, 'band': 'ssm', 'system:index': '2'}},
  {'type': 'Image',
   'bands': [{'id': 'ssm',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'month':

In [ ]:
smap_asset = ee.data.createAsset({'type': 'ImageCollection'}, 'users/jamesmcc/smap_usda_climatology')

In [ ]:
smap_asset

In [82]:
months = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
bands = ['smp'] #, 'ssm', 'susm']
#bands = ['porosity', 'avail_porosity']
bbox = ee.Geometry.BBox(-180.0, -90, 180, 90)
for bb in bands:
    for mm0 in range(12):
        mm = mm0 + 1
        description = 'SMAP-USDA_mon_clim_' + str(bb) + '_' + str(mm).zfill(2) + '_' + months[mm0]
        xx = ee.batch.Export.image.toAsset(
            smap_collection
            .filter(ee.Filter.eq('month', mm))
            .filter(ee.Filter.eq('band', bb)).first(),
            description=description, 
            assetId=smap_asset['id'] + '/' + description,
            region=bbox).start()